In [1]:
import sys
import os
import json
import pandas as pd

# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
from hybridtablerag.storage.duckdb_manager import DuckDBManager

# Persistent DB
db = DuckDBManager(db_path="data/hybridtablerag.duckdb")

# Register CSV
db.register_csv("../data/sales.csv", table_name="sales")
db.register_csv("../data/customers.csv", table_name="customers")

# List tables
db.list_tables()

# Inspect schema
print(json.dumps(db.get_table_schema("sales"), indent=4))
print(json.dumps(db.get_table_schema("customers"), indent=4))


[DuckDB] Initialized at d:\Nikita\AI ML Engineer\s2s dynamics\HybridTableRag\data/hybridtablerag.duckdb
[DuckDB] Registering table: sales
[DuckDB] Registering table: customers
[
    {
        "column_name": "order_id",
        "data_type": "BIGINT",
        "null": "YES",
        "key": null,
        "default": null,
        "extra": null
    },
    {
        "column_name": "customer_id",
        "data_type": "BIGINT",
        "null": "YES",
        "key": null,
        "default": null,
        "extra": null
    },
    {
        "column_name": "product",
        "data_type": "VARCHAR",
        "null": "YES",
        "key": null,
        "default": null,
        "extra": null
    },
    {
        "column_name": "category",
        "data_type": "VARCHAR",
        "null": "YES",
        "key": null,
        "default": null,
        "extra": null
    },
    {
        "column_name": "region",
        "data_type": "VARCHAR",
        "null": "YES",
        "key": null,
        "default": null

In [12]:
# Run SQL
print(display(pd.DataFrame(db.execute_query("SELECT COUNT(*) FROM sales"))))
print(display(pd.DataFrame(db.execute_query("SELECT SUM(quantity * unit_price) AS total_revenue FROM sales;"))))
print(display(pd.DataFrame(db.execute_query("SELECT region, SUM(quantity * unit_price) AS revenue FROM sales GROUP BY region;"))))
print(display(pd.DataFrame(db.execute_query("SELECT * FROM sales WHERE order_date >= '2023-03-01';"))))

print(display(pd.DataFrame(db.execute_query("""SELECT c.name, SUM(s.quantity * s.unit_price) AS revenue
FROM sales s
JOIN customers c
ON s.customer_id = c.customer_id
GROUP BY c.name;"""))))

print(display(pd.DataFrame(db.execute_query("""
SELECT c.industry, SUM(s.quantity * s.unit_price) AS revenue
FROM sales s
JOIN customers c
ON s.customer_id = c.customer_id
GROUP BY c.industry
ORDER BY revenue DESC;
"""))))

,count_star()
0,5


None


,total_revenue
0,3850.0


None


,region,revenue
0,North,1900.0
1,West,600.0
2,East,750.0
3,South,600.0


...

None


,name,revenue
0,Alice Corp,1900.0
1,Charlie Inc,600.0
2,Delta LLC,750.0
3,Bravo Ltd,600.0


None


,industry,revenue
0,Technology,1900.0
1,Retail,750.0
2,Finance,600.0
3,Media,600.0


None


In [14]:
db.get_database_schema_overview()

{'customers': {'columns': [{'column_name': 'customer_id',
    'data_type': 'BIGINT',
    'null': 'YES',
    'key': None,
    'default': None,
    'extra': None},
   {'column_name': 'name',
    'data_type': 'VARCHAR',
    'null': 'YES',
    'key': None,
    'default': None,
    'extra': None},
   {'column_name': 'industry',
    'data_type': 'VARCHAR',
    'null': 'YES',
    'key': None,
    'default': None,
    'extra': None},
   {'column_name': 'country',
    'data_type': 'VARCHAR',
    'null': 'YES',
    'key': None,
    'default': None,
    'extra': None},
   {'column_name': 'signup_date',
    'data_type': 'DATE',
    'null': 'YES',
    'key': None,
    'default': None,
    'extra': None}]},
 'sales': {'columns': [{'column_name': 'order_id',
    'data_type': 'BIGINT',
    'null': 'YES',
    'key': None,
    'default': None,
    'extra': None},
   {'column_name': 'customer_id',
    'data_type': 'BIGINT',
    'null': 'YES',
    'key': None,
    'default': None,
    'extra': None},
   {

In [6]:
relationships = db.infer_relationships()
print(json.dumps(relationships, indent=4))

{
    "customers": [
        {
            "from_column": "customer_id",
            "to_table": "sales",
            "to_column": "customer_id",
            "data_type": "BIGINT"
        }
    ],
    "sales": [
        {
            "from_column": "customer_id",
            "to_table": "customers",
            "to_column": "customer_id",
            "data_type": "BIGINT"
        }
    ]
}


In [4]:
print(db.suggest_joins(["sales", "customers"]))

['sales.customer_id = customers.customer_id']
